In [ ]:
# 한글 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [1]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요 모듈 설치
!pip install konlpy

In [4]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from konlpy.tag import Okt
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# data 폴더 설정
fdir='/content/drive/MyDrive/deu2/data/'

In [6]:
okt=Okt()

# 원본 데이터 읽어오기
data = pd.read_csv(fdir+'부산관광지_up.csv')

# title에 대한 TF 가져오기
with open(fdir + 'title_tf.pickle', 'rb') as f:
    title_tf = pickle.load(f)

# title의 IDF 가져오기
with open(fdir + 'title_idf.pickle', 'rb') as f:
    title_idf = pickle.load(f)

# title의 TF-IDF 가져오기
with open(fdir + 'title_tfidf.pickle', 'rb') as f:
    title_tfidf = pickle.load(f)


In [25]:
new_text='부산 가볼만한 관광지 추천해주세요.'

new_nouns=okt.nouns(new_text) # new_text에서 명사 추출

# 새로 나타난 토픽(단어) 데이터 제거하기
new_tf=pd.Series(Counter(new_nouns)) # new_text에 대한 TF 계산
new_text_filter=[ i for i in new_tf.index if i in list(title_idf.index) ]

# 최종 TF 계산
new_tf=pd.Series(Counter(new_text_filter))

In [39]:
# 새로 입력한 글에 대한 TF-IDF 계산
new_tfidf = new_tf*title_idf
new_tfidf=new_tfidf.fillna(0)
new_tfidf=new_tfidf.values.reshape(1, -1)
new_tfidf.shape

(1, 399)

In [45]:
# 새로운 글과 가장 유사한 원본 데이터 찾기
# cosine_similarity: https://wikidocs.net/24603 참조
sim_res=[]
for i in title_tfidf.index:
    sim_res.append(cosine_similarity(new_tfidf, title_tfidf.loc[i].values.reshape(1,-1)).flatten().tolist())

sim_res[:5]

[[0.814896550087482],
 [0.1779392707258571],
 [0.1779392707258571],
 [0.24305583556802854],
 [0.28622778008226846]]

In [48]:
# enumerate(나열형데이터): index, 값을 동시에 반환
# 코사인 유사도 결과에 대한 리스트 생성
sim_res_dict=[]
for idx, i in enumerate(sim_res): 
    sim_res_dict.append((idx, i[0]))

len(sim_res_dict)

1420

In [54]:
## 원본 데이터에서 유사도가 높은 30개 글 추출(유사도로 내림차순 정렬후 상위 30개 추출)
sim30_sort=sorted(sim_res_dict, key=lambda x: x[1], reverse=True)[:30]
sim30_sort_index=[i[0] for i in sim30_sort]  # 상위 30개에 대한 index 값
sim30_sort_score=[i[1] for i in sim30_sort]  # 상위 30개에 대한 유사도 점수

# 작업 내용 저장
#파일이름(한글,숫자, 영어)로 제한, 특수문자 제외
filename=' '.join(re.findall('[가-힣0-9a-zA-Z]+', new_text))
data.loc[sim30_sort_index].to_csv(f'{fdir}title_similarity_30_{filename}.csv')
print(f'{fdir}에 저장되었습니다.')

/content/drive/MyDrive/deu2/data/에 저장되었습니다.


In [65]:
class Similarity():
    def __init__(self, new_text):
        self.new_text=new_text

    def view_Similarity(self):
        # 사용자가 새로 입력한 텍스트에 대한 TF 구하기
        self.new_nouns=okt.nouns(self.new_text) # 사용가 입력한 new_text에서 명사 추출
        self.new_tf=pd.Series(Counter(self.new_nouns)) # new_text에 대한 TF 계산

        # 새로 나타난 토픽(단어) 데이터 제거하기
        self.new_tf=pd.Series(Counter(self.new_nouns)) # new_text에 대한 TF 계산
        self.new_text_filter=[ i for i in self.new_tf.index if i in list(title_idf.index) ]

        # 최종 TF 계산
        self.new_tf=pd.Series(Counter(self.new_text_filter))

        # 새로 입력한 글에 대한 TF-IDF 계산
        self.new_tfidf = self.new_tf*title_idf
        self.new_tfidf=self.new_tfidf.fillna(0)
        self.new_tfidf=self.new_tfidf.values.reshape(1, -1)
        self.new_tfidf.shape

        # 새로운 글과 가장 유사한 원본 데이터 찾기
        # cosine_similarity: https://wikidocs.net/24603 참조
        self.sim_res=[cosine_similarity(self.new_tfidf, title_tfidf.loc[i].values.reshape(1,-1)).flatten().tolist() for i in title_tfidf.index]
    
        # enumerate(나열형데이터): index, 값을 동시에 반환
        # 코사인 유사도 결과에 대한 리스트 생성
        self.sim_res_dict=[(idx, i[0]) for idx, i in enumerate(self.sim_res)]

        ## 원본 데이터에서 유사도가 높은 30개 글 추출(유사도로 내림차순 정렬후 상위 30개 추출)
        self.sim30_sort=sorted(self.sim_res_dict, key=lambda x: x[1], reverse=True)[:30]
        self.sim30_sort_index=[i[0] for i in self.sim30_sort]  # 상위 30개에 대한 index 값
        self.sim30_sort_score=[i[1] for i in self.sim30_sort]  # 상위 30개에 대한 유사도 점수

        # 작업 내용 저장
        #파일이름(한글,숫자, 영어)로 제한, 특수문자 제외
        filename=' '.join(re.findall('[가-힣0-9a-zA-Z]+', self.new_text))
        data.loc[self.sim30_sort_index].to_csv(f'{fdir}title_similarity_30_{filename}.csv')
        print(f'{fdir}에 저장되었습니다.')

        # 새로운 글에 대한 유사 문자 3개 출력하기
        self.df_sim=data.loc[self.sim30_sort_index[:3]]
        print('=================================================')
        print(f'   >>>  질문내용(Q){self.new_text}')
        print('=================================================')

        for idx, i in enumerate(self.df_sim.index):
            print('==============================================')
            print('날짜', data.loc[i, 'ques_date'])
            print('제목', data.loc[i, 'title'])
            print('유사도', self.sim30_sort_score[idx])
            print('질문', data.loc[i, 'question'])
            print('답변', data.loc[i, 'answer'])
            print('\n\n')
           


In [66]:
if __name__ == '__main__':
    Q=input('검색어 입력:')
    sim_text=Similarity(Q)  # 유사도 추출 객체 생성
    sim_text.view_Similarity()

검색어 입력:해운대 관광지 추천 부탁드립니다.
/content/drive/MyDrive/deu2/data/에 저장되었습니다.
   >>>  질문내용(Q)해운대 관광지 추천 부탁드립니다.
날짜 2023.01.12
제목      부산 해운대 추천   
유사도 0.8926732900226786
질문  내일 오전일찍 출발해서 오후1시쯤 부산역 도착예정입니다. 일요일이나 월요일까지 방문할생각중인데 어디어디가면 좋을까요?꼭 부산이나 해운대 아니여도 근방 좋은곳도 추천받습니다.자차 안몰고 대중교통탑니다!숙소는 그때그때 게스트하우스같은곳에서 지낼 예정입니다! 
답변      부산여행 필요정보에 도움이 되고자 나름 자료들을 정리한 홈페이지 참고하세요 ..쉬운여행 https://myeasytrip.modoo.at/ 홈 | 쉬운여행 (modoo.at)​부산여행 2박3일 즐기기 추천일정 참고적으로 보세요 ...​1일차는 태종대, 송도해수욕장 과 남포동, 광복동, 부평야시장등 먹거리와 야경즐기기​2일차는 해동용궁사와 해운대, 광안리 해수욕장 관광, 오륙도 이기대공원,아쿠아리움, 동백섬, 유람선선착장, 블루라인파크 해변열차​3일차는 용두산공원이나 롯데백화점 11층 스카이파크 에서 보는 부산의 스카이라인,그리고 자갈치시장, 자갈치전망대, 국제시장 쇼핑 과 광복동 거리구경 ​추천관광지와 일정순서등은 숙박지 위치와 관광당일 출발시간이나 여러상황 등에서편리한데로 일정순서를 바꾸거나 관광지 순서를 바꾸어도 되네요 .. ​네이버지도에서 발췌한 부산전도를 참고하여 부산의 관광지 위치를 파악하세요.                    추천일정의 1일차는태종대, 부산의 대표적인 관광지로 영도의 끝자락에 위치하며부산역에서 버스, 택시등으로 편도 40, 50분 정도 소요되며,태종대를 한바퀴 돌아보면서 부산의 앞바다를 조망하는 관광은개인에 따라 차이가 있겠으나 1시간이상, 2,3시간 정도를 생각하여야 하고                                태종대의 자갈마당에서는 유람선을 이용하여 해상관